In [1]:
import peft
import transformers
import torch
import numpy as np

print(peft.__version__)

[2023-08-24 16:17:00,103] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
0.5.0


In [2]:
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")

In [3]:
peft_config =  dict(
    inference_mode=False,
    lora_dropout=0.,
    lora_alpha=16,
    bias="none",
    r=16,
)

In [4]:
peft_model: peft.PeftModel = peft.get_peft_model(
    model=model, 
    peft_config=peft.LoraConfig(**peft_config), 
    adapter_name="policy",
)

/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:
print(f"{type(peft_model) = }")
print(f"{len(peft_model.peft_config) = }")
print(type(peft_model.base_model))
print(type(peft_model.peft_config))
print(peft_model.peft_config)
print(peft_model.base_model.peft_config)
print(peft_model.base_model.peft_config is peft_model.peft_config)

peft_model.add_adapter(
    adapter_name="value", 
    peft_config=peft.LoraConfig(**peft_config),
)

print(peft_model.base_model.peft_config)
print(peft_model.base_model.peft_config is peft_model.peft_config)

peft.LoraModel.disable_adapter_layers

type(peft_model) = <class 'peft.peft_model.PeftModel'>
len(peft_model.peft_config) = 2
<class 'peft.tuners.lora.LoraModel'>
<class 'dict'>
{'policy': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='gpt2', revision=None, task_type=None, inference_mode=False, r=16, target_modules=['c_attn'], lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=True, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None), 'value': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=16, target_modules=['c_attn'], lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)}
{'policy': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='gpt2', revision=None, task_type=None, inference_mode=

'policy'

In [25]:
print(peft_model.active_adapter)
peft_model.set_adapter("value")
print(peft_model.active_adapter)

def check_is_disabled(model):
    assert isinstance(model, peft.PeftModel)
    assert isinstance(model.base_model, peft.LoraModel)
    
    values = []
    for layer in model.base_model.modules():
        if isinstance(layer, peft.tuners.lora.LoraLayer):
            values.append(layer.disable_adapters)
        elif isinstance(layer, peft.tuners.lora.ModulesToSaveWrapper):
            values.append(layer.disable_adapters)
            
    return all(values)


with peft_model.disable_adapter():
    print("check_is_disabled", check_is_disabled(peft_model))
    print("peft_model.active_adapter", peft_model.active_adapter)
    peft_model.set_adapter("policy")
    print("check_is_disabled", check_is_disabled(peft_model))
print("check_is_disabled", check_is_disabled(peft_model))

policy
value
check_is_disabled True
peft_model.active_adapter value
check_is_disabled True
check_is_disabled False


In [6]:
model_1 = transformers.GPT2LMHeadModel(config=transformers.AutoConfig.from_pretrained("gpt2"))

In [7]:
model_1.parameters = model.parameters

In [8]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_convert_head_mask_to_5d',
 '_create_repo',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_from_config',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_decoder_start_token_id',
 '_get_files_timestamps',
 '_get_logits_processor',
 '_get_logits_warpe

In [9]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (policy): Identity()
                  (value): Identity()
                )
                (lora_A): ModuleDict(
                  (policy): Linear(in_features=768, out_features=16, bias=False)
                  (value): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (policy): Linear(in_features=16, out_features=2304, bias=False)
                  (value): Linear(in_featur